In [11]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

from model import AGC_STFT_GRU

from torchsummary import summary

model = AGC_STFT_GRU(160, 40, 160, 80).to('cuda:0')
summary(model, (160000,), batch_size=1)

print(sum(p.numel() for p in model.parameters()))
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               GRU-1  [[-1, 2000, 40], [-1, 2, 40]]               0
              ReLU-2              [1, 2000, 40]               0
            Linear-3               [1, 2000, 1]              41
          Upsample-4             [1, 1, 160000]               0
Total params: 41
Trainable params: 41
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.61
Forward/backward pass size (MB): 50.67
Params size (MB): 0.00
Estimated Total Size (MB): 51.28
----------------------------------------------------------------
14801
14801


In [7]:
from model import AGC_STFT_GRU
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import time
import soundfile as sf
import numpy as np

model = AGC_STFT_GRU(160, 40, 160, 80)   
package = torch.load('/home/yhjeon/projects/IITP_SE/NS_AGC/models/AGC_endtoend_r1/trained_ckpts/default.pth.tar', map_location='cpu')
model.load_state_dict(package['state_dict'])

model.cpu()
model.eval()

wav = sf.read('/home/yhjeon/projects/IITP_SE/NS_AGC/models/AGC_endtoend_r1/samples/wave_distorted_{0}.wav'.format(0))[0]
wav = torch.FloatTensor(wav)
wav = wav.unsqueeze(0).cpu()

frame = 100
wav_frame = wav[...,(frame)*80:(frame + 2)*80]
wav_frame2 = torch.cat((torch.zeros((1, 80)), wav[...,(frame)*80:(frame + 2)*80]), dim=1)

wav_frame_stft = torch.stft(wav_frame, n_fft=160, hop_length=80, win_length=160, center=False, window=torch.hann_window(160, periodic=True).to('cpu'), return_complex=False)
wav_frame2_stft = torch.stft(wav_frame2, n_fft=160, hop_length=80, win_length=160, center=False, window=torch.hann_window(160, periodic=True).to('cpu'), return_complex=False)

print(wav_frame_stft.shape)
print(wav_frame2_stft.shape)

print(wav_frame_stft[:, :, -1, :] - wav_frame2_stft[:, :, -1, :])


torch.Size([1, 81, 1, 2])
torch.Size([1, 81, 2, 2])
tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]